In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import cloudscraper
import warnings
warnings.filterwarnings("ignore")

In [2]:
stories_urls_df = pd.read_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Selected_Sections_data.csv")

In [3]:
abroad_gh = stories_urls_df[stories_urls_df['Section Title'] == "Abroad, Ghanaians"]

## Getting story URLS

In [4]:
df_columns = ['Section Title','Url','Url Extracted']

In [5]:
def get_story_urls(row):
    try:
        df = pd.DataFrame()
        scraper = cloudscraper.create_scraper()
        page = scraper.get(row["Url"])
        soup = BeautifulSoup(page.content, 'html.parser')
        page_stories = soup.find_all("div", class_ = "afcon-news list")
        for ul in page_stories:
            for li in ul.findAll('li'):
                data = li.find("a")
                url = row['Url']+data.get('href')
                url_extracted = 0
                temp_row = pd.DataFrame(data = [[row["Section Title"], url,url_extracted]], columns = df_columns)
                df = pd.concat([df,temp_row], ignore_index=True)  
            time.sleep(20)
    except:
        pass

    return df

In [6]:
del abroad_gh['Unnamed: 0']

In [7]:
url_df  = abroad_gh.apply(lambda row : get_story_urls(row),axis = 1)

In [9]:
article_urls_df = pd.DataFrame(columns = df_columns)

In [10]:
for row in url_df:
    article_urls_df = article_urls_df.append(row)

In [17]:
article_urls_df.to_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Abroad Ghanaians Urls.csv")

In [12]:
##push to db

## Extracting Stories from urls

In [21]:
story_df_columns = ['Section','Header','Date','Title','Information']
stories_data = pd.DataFrame(columns = story_df_columns)

In [22]:
def get_abroad_gh_articles(section_title,url):
    df = pd.DataFrame()
    try:
        scraper = cloudscraper.create_scraper()
        page = scraper.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        page_stories = soup.find(id = "medsection1")
        div_class = page_stories.find("div", class_ = "article-left-col")
        header = div_class.find('p', class_ = "floatLeft").text
        date = div_class.find('a', id = "date").text
        title = div_class.find('h1').text
        info = div_class.find('p',id = "article-123").text
        temp_row = pd.DataFrame(data = [[section_title, header, date, title, info]], columns = story_df_columns)
        #stories_data = stories_data.append(temp_row)
        df = pd.concat([df,temp_row],ignore_index =True)
    except:
        file_object = open("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\Error Urls\\article urls.txt", 'a')
        file_object.write(f'\n\n{section_title},{url}')
        file_object.close()
    return df

In [23]:
abroad_gh_final_df = article_urls_df.apply(lambda row:get_abroad_gh_articles(row['Section Title'],row['Url']), axis = 1)

In [24]:
abroad_gh_final_df

0                  Section                          ...
1                 Empty DataFrame
Columns: []
Index: []
2                 Empty DataFrame
Columns: []
Index: []
3                 Empty DataFrame
Columns: []
Index: []
4                  Section                          ...
5                 Empty DataFrame
Columns: []
Index: []
6                  Section                          ...
7                 Empty DataFrame
Columns: []
Index: []
8                  Section                          ...
9                  Section                          ...
10                 Section                          ...
11                Empty DataFrame
Columns: []
Index: []
12                 Section                          ...
13                 Section                          ...
14                 Section                          ...
15                 Section                          ...
16                 Section                          ...
17                 Section                      